<a id = "head"></a>
# <p style="text-align: center;">Исследование надёжности заёмщиков</p>
---

### Описание проекта 

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

### План выполнения проекта:
***

1. [Изучение общей информации о данных](#step1)
2. [Предобработка данных. Категоризация.](#step2)
3. [Ответы на вопросы проекта](#step3)
    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?
4. [Общий вывод по проекту](#step4)

<a id = "step1"></a>
## Шаг 1. Изучение общей информации о данных 

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [7]:
data[data['dob_years'] < 18]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

---
Найдены пропуски в столбцах "days_employed" и "total_income". Также в столбце "days_employed" много отрицательных значений, чего в действительности быть не может. Тип данных в этом столбце float64, что тоже неправильно.

Аномалии в данных:
1. Столбец "gender": нашел одного человека с непонятным полом;
2. Столбец "сhildren": у 47 человек -1 ребенок, у 76 человек 20 детей (тоже маловероятно);
3. Столбец "dob_years: у 101 заемщика указан возраст 0.

<a id = "step2"></a>
## Шаг 2. Предобработка данных. Категоризация.

[Начало](#head) [1](#step1) [3](#step3) [4](#step4)

### Обработка пропусков

In [8]:
print('Количество пропусков в столбце "ежемесячный доход": {}'.format(data['total_income'].isna().sum()))

data = data.drop(['days_employed'], axis = 1)

Количество пропусков в столбце "ежемесячный доход": 2174


In [9]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [10]:
income_type_median = data.groupby('income_type')['total_income'].median()

for income in data['income_type']:
    data.loc[(data['total_income'].isnull()) & (data['income_type'] == income), 'total_income']\
                = income_type_median[income]

---

Пропуски были найдены в столбцах "days_employed" и "total_income", данные относятся к количественным 
значениям.
В обоих столбцах обнаружено по 2174 пропуска. Все пропущенные значения в двух столбцах полностью совпадают у одних и тех же людей. Они могут быть вызваны либо ошибкой выгрузки данных, либо по этим заемщикам в принципе отсутсвует какая-либо информация об их стаже и ежемесячном доходе. Необходимо обратить внимание разработчиков на данные пропуски.
Так как в данных со стажем содержится много ошибок и эти показатели не сильно влияют на результаты исследования, я решил удалить столбец "days_employed" из таблицы.
Пропущенные значения в столбце "total_income" я заполнял медианными значениями в зависимости от типа занятости, чтобы увеличить точность заполнения.

### Замена типа данных

In [11]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Обработка дубликатов

In [12]:
data['education'] = data['education'].str.lower()
print('Дубликатов в таблице:', data.duplicated().sum())
print()

data = data.drop_duplicates().reset_index(drop = True)

Дубликатов в таблице: 71



---
Всего был выявлен 71 дубликат. Возможные причины появления дубликатов: ошибки при выгрузке, неправильное соединение данных из разных источников, ошибки пользователя при занесении информации.

Все дубликаты были удалены из таблицы.

### Дополнительная обработка данных 

In [13]:
data = data.drop(data[(data['children'] == (-1))|(data['children'] == 20)].index).reset_index(drop = True)

In [14]:
data.shape

(21331, 11)

---
Поскольку в исходных данных присутсвуют клиенты, у которых по 20 детей и один клиент с -1 ребенком, я решил что будет лучше исключить их из исследования. Их доля очень маленькая, на результаты исследования мое решение повлиять не должно. 

### Лемматизация

In [15]:
m = Mystem()
# data['purpose'].value_counts()
purposes_dict = ['недвижимость', 'жилье', 'свадьба', 'автомобиль', 'образование']

In [16]:
def find_purpose(value):
    
    for purpose in purposes_dict:
        if purpose in m.lemmatize(value):
            return purpose

In [17]:
data['new_purpose'] = data['purpose'].apply(find_purpose)

In [18]:
data['new_purpose'].value_counts()

недвижимость    6314
жилье           4437
автомобиль      4279
образование     3988
свадьба         2313
Name: new_purpose, dtype: int64

---
Вручную был составлен справочник целей получения кредита. Лемматизация происходит внутри функции. Если цель из моего справочника находит соответсвующее значение в лемматизированной строке из столбца 'purpose', то функция возвращает новое значение из справочника.

### Категоризация данных

In [19]:
def children(kids):
    if kids == 0:
        return 'детей нет'
    return 'дети есть'

data['children_group']  = data['children'].apply(children)    

In [20]:
def kids_number(number):
    if number == 0:
        return 'детей нет'
    if number == 1:
        return 'один ребенок'
    return 'два и более детей'

data['children_number_group'] = data['children'].apply(kids_number)

In [21]:
data['total_income'].mean()

165342.74576906848

In [22]:
def income_group(income):
    if income < 75000:
        return 'доход до 75 тыс.руб.'
    if income < 125000:
        return 'доход до 125 тыс.руб.'
    if income < 150000:
        return 'доход от 100 до 150 тыс.руб.'
    if income < 200000:
        return 'доход от 150 до 200 тыс.руб.'
    if income < 250000:
        return 'доход от 200 до 250 тыс.руб.'
    return 'доход свыше 250 тыс.руб.'

data['income_group'] = data['total_income'].apply(income_group)

In [23]:
data['income_group'].value_counts()

доход до 125 тыс.руб.           5862
доход от 150 до 200 тыс.руб.    4738
доход от 100 до 150 тыс.руб.    3838
доход свыше 250 тыс.руб.        2797
доход от 200 до 250 тыс.руб.    2242
доход до 75 тыс.руб.            1854
Name: income_group, dtype: int64

In [24]:
def age_group(age):
    if age == 0:
        return 'нет данных'
    if age < 30:
        return 'младше 30 лет'
    if age < 40:
        return '30 - 40 лет'
    if age < 50:
        return '40 - 50 лет'
    if age < 65:
        return '50 - 60 лет'
    return 'старше 65 лет'

data['age_group'] = data['dob_years'].apply(age_group)

In [25]:
data

children  dob_years education  education_id     family_status  \
0             1         42    высшее             0   женат / замужем   
1             1         36   среднее             1   женат / замужем   
2             0         33   среднее             1   женат / замужем   
3             3         32   среднее             1   женат / замужем   
4             0         53   среднее             1  гражданский брак   
...         ...        ...       ...           ...               ...   
21326         1         43   среднее             1  гражданский брак   
21327         0         67   среднее             1   женат / замужем   
21328         1         38   среднее             1  гражданский брак   
21329         3         38   среднее             1   женат / замужем   
21330         2         40   среднее             1   женат / замужем   

       family_status_id gender income_type  debt  total_income  \
0                     0      F   сотрудник     0        253875   
1                     0      F   сотрудник     0        112080   
2                     0      M   сотрудник     0        145885   
3                     0      M   сотрудник     0        267628   
4                     1      F   пенсионер     0        158616   
...                 ...    ...         ...   ...           ...   
21326                 1      F   компаньон     0        224791   
21327                 0      F   пенсионер     0        155999   
21328                 1      M   сотрудник     1         89672   
21329                 0      M   сотрудник     1        244093   
21330                 0      F   сотрудник     0         82047   

                            purpose   new_purpose children_group  \
0                     покупка жилья         жилье      дети есть   
1           приобретение автомобиля    автомобиль      дети есть   
2                     покупка жилья         жилье      детей нет   
3        дополнительное образование   образование      дети есть   
4                   сыграть свадьбу       свадьба      детей нет   
...                             ...           ...            ...   
21326             операции с жильем         жилье      дети есть   
21327          сделка с автомобилем    автомобиль      детей нет   
21328                  недвижимость  недвижимость      дети есть   
21329  на покупку своего автомобиля    автомобиль      дети есть   
21330         на покупку автомобиля    автомобиль      дети есть   

      children_number_group                  income_group      age_group  
0              один ребенок      доход свыше 250 тыс.руб.    40 - 50 лет  
1              один ребенок         доход до 125 тыс.руб.    30 - 40 лет  
2                 детей нет  доход от 100 до 150 тыс.руб.    30 - 40 лет  
3         два и более детей      доход свыше 250 тыс.руб.    30 - 40 лет  
4                 детей нет  доход от 150 до 200 тыс.руб.    50 - 60 лет  
...                     ...                           ...            ...  
21326          один ребенок  доход от 200 до 250 тыс.руб.    40 - 50 лет  
21327             детей нет  доход от 150 до 200 тыс.руб.  старше 65 лет  
21328          один ребенок         доход до 125 тыс.руб.    30 - 40 лет  
21329     два и более детей  доход от 200 до 250 тыс.руб.    30 - 40 лет  
21330     два и более детей         доход до 125 тыс.руб.    40 - 50 лет  

[21331 rows x 16 columns]

Были созданы дополнительные `4` категории:
1. children_group - поделил заемщиков на тех у кого дети есть и у кого их нет (Хочу посмотреть в целом разницу по просрочкам между двумя этими группами);
2. children_number_group - категории в зависимости от количества детей (Эта классификация сделана, чтобы увидеть влияет ли количество детей на возврат кредита в срок);
3. income_group - разбил заемщиков на группы в зависимости от уровня дохода;
4. age_group - возрастные группы.

Данные группы помогут ответить на вопросы исследования.

<a id = "step3"></a>
## Шаг 3. Ответы на вопросы проекта

[Начало](#head) [1](#step1) [2](#step2) [4](#step4)

### 1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
---

In [26]:
data['debt'].mean()

0.08119638085415593

***Из всей выборки 8,12% имели задолженность по возврату кредитов***

In [27]:
data.pivot_table(index='children_group', values=['debt'], aggfunc=['count','mean'])

count      mean
                 debt      debt
children_group                 
детей нет       14091  0.075438
дети есть        7240  0.092403

In [28]:
data.pivot_table(index='children_number_group', values=['debt'], aggfunc=['count','mean'])

count      mean
                        debt      debt
children_number_group                 
два и более детей       2432  0.092516
детей нет              14091  0.075438
один ребенок            4808  0.092346

---
На основании полученных данных можно сказать, что зависимость между наличием детей и возвратом кредита в срок есть. В среднем 7,5% заемщиков без детей имели задолженность по кредиту. У клиентов, которых есть дети, эта доля составляет 9,2%. Таким образом клиенты с детьми с большей вероятностью могут не вернуть кредит в срок. Видимо, заемщикам с детьми сложнее планировать свои финансы. При этом количество детей не сказывается на невозврат. 

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
---

In [29]:
data.pivot_table(index='family_status', values=['debt'], aggfunc=['count','mean'])

count      mean
                        debt      debt
family_status                         
Не женат / не замужем   2796  0.097639
в разводе               1189  0.070648
вдовец / вдова           951  0.066246
гражданский брак        4134  0.093130
женат / замужем        12261  0.075606

In [30]:
data.pivot_table(index='family_status', columns = 'children_group', values=['debt'], aggfunc=['count','mean'])

count                mean          
                           debt                debt          
children_group        детей нет дети есть детей нет дети есть
family_status                                                
Не женат / не замужем      2262       534  0.092838  0.117978
в разводе                   784       405  0.070153  0.071605
вдовец / вдова              847       104  0.062574  0.096154
гражданский брак           2730      1404  0.083883  0.111111
женат / замужем            7468      4793  0.069095  0.085750

---
Между семейным положением и возвратом кредита в срок также наблюдается зависимость. Люди, состоящие в браке, являются более дисциплинированными заемщиками, среди них меньше процент просрочек - 7,6%. Среди тех, кто не регистрировал свои отношения официально (это касается и тех, кто живет в гражданском браке), доля заемщиков с задолженностями по кредиту превышает 9%. Примечательно, что клиенты, которые когда-то были в браке, также остаются ответсвенными в отношении возврата долга.

Вторая таблица дополнительно подтверждает выводы, сформудированные в ответе на первый вопрос исследования.

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
---

In [31]:
data.pivot_table(index=['income_group'], values=['debt'], aggfunc=['count','mean'])

count      mean
                              debt      debt
income_group                                
доход до 125 тыс.руб.         5862  0.086319
доход до 75 тыс.руб.          1854  0.073355
доход от 100 до 150 тыс.руб.  3838  0.086243
доход от 150 до 200 тыс.руб.  4738  0.085057
доход от 200 до 250 тыс.руб.  2242  0.072257
доход свыше 250 тыс.руб.      2797  0.069360

In [32]:
data['amount_group'] = pd.cut(data['total_income'],[0,150000,10000000])

In [33]:
data.groupby('amount_group')['debt'].agg(['count','mean'])

count      mean
amount_group                       
(0, 150000]         11554  0.084213
(150000, 10000000]   9777  0.077631

---
Если разбить всех клиентов по уровню дохода на две группы, то можно сделать вывод, что более богатые клиенты являются более надежными заемщиками. 
Однако, если выделить среди клиентов банка больше групп по уровню дохода, открывается интересный факт. У клиентов с низким доходом (относительно всей группы) процент тех, кто возвращал кредит с просрочкой (7,3%) ниже среднего показателя по всей выборке. 
Отвечая на поставленный вопрос, можно сказать, что  зависимость между уровнем дохода и возвратом кредита в срок есть. Клиенты банка со средними доходами более склонны к невозврату кредита в срок, в то время как заемщики с низкими и высокими доходами являются более дисциплинированными. 

### 4. Как разные цели кредита влияют на его возврат в срок?
---

In [34]:
data.pivot_table(index='new_purpose', values=['debt'], aggfunc=['count','mean'])

count      mean
              debt      debt
new_purpose                 
автомобиль    4279  0.093480
жилье         4437  0.069416
недвижимость  6314  0.074755
образование   3988  0.092528
свадьба       2313  0.079118

---
Самые надежные заемщики те, кто брал кредит на цели связанные с жильем или недвижимостью. 
У тех клиентов, кто брал кредит на свадьбу доля невозвратов ниже средней по всей выборке.

In [35]:
data['purpose_new_cut'] = data['new_purpose'].replace('жилье', 'недвижимость')

In [36]:
data.groupby('purpose_new_cut')['debt'].agg(['count','mean'])

count      mean
purpose_new_cut                 
автомобиль        4279  0.093480
недвижимость     10751  0.072551
образование       3988  0.092528
свадьба           2313  0.079118

---
Так как цели связанные с жильем и недвижимостью близкие по смыслу, я решил объединить их в одну группу. 

In [37]:
data.pivot_table(index='age_group', columns='purpose_new_cut' , values=['debt'], aggfunc=['count','mean'])

count                                        mean  \
                      debt                                        debt   
purpose_new_cut автомобиль недвижимость образование свадьба автомобиль   
age_group                                                                
30 - 40 лет           1130         2835        1038     622   0.118584   
40 - 50 лет           1080         2692         972     574   0.083333   
50 - 60 лет           1267         3114        1185     663   0.068666   
младше 30 лет          596         1628         601     341   0.127517   
нет данных              21           47          18      14   0.142857   
старше 65 лет          185          435         174      99   0.054054   

                                                    
                                                    
purpose_new_cut недвижимость образование   свадьба  
age_group                                           
30 - 40 лет         0.083598    0.112717  0.096463  
40 - 50 лет         0.069837    0.083333  0.073171  
50 - 60 лет         0.052987    0.070886  0.061840  
младше 30 лет       0.100123    0.129784  0.093842  
нет данных          0.063830    0.000000  0.142857  
старше 65 лет       0.055172    0.051724  0.060606

---
Даже в разрезе по возрастам цели кредитов, связанные с недвижимостью и свадьбой выглядят лучше на фоне остальных.

Среди людей, которые брали кредит на образование или автомобиль было больше тех, кто возвращал долг с просрочкой. Кредиты на цели связанные с недвижимостью или с организацией свадеб возвращали более ответственно и в срок. 

<a id = "step4"></a>
## Шаг 4. Общий вывод по проекту

[Начало](#head) [1](#step1) [2](#step2) [3](#step3)

Вопросы исследования:

    - Есть ли зависимость между наличием детей и возвратом кредита в срок?
    - Есть ли зависимость между семейным положением и возвратом кредита в срок?
    - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    - Как разные цели кредита влияют на его возврат в срок?

В ходе исследования было установлено, что имееются определенные зависимости между наличием детей, семейным положением, уровнем дохода и возвратом кредита в срок. Заемщикам без детей было проще платить по кредитам вовремя. 
Семейное положение тоже сказывается на обслуживании кредита: клиенты, состоящие в браке, более дисциплинированы.
Интересная зависимость выявлена между клиентами с разными доходами. Заемщики с низким и высоким заработком более ответсвенно подходят к возврату кредита в срок.
При рассмотрении зависимости целей кредита от его возврата в срок было установлено, что меньше просрочек у заемщиков, которые брали кредит на цели, связанные с недвижимостью или с организацией свадьбы.